## Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from google.cloud import bigquery
from pathlib import Path
import sys

sys.path.insert(0, "../")
from params import *

sys.path.insert(0, '../custom_functions')
from get_data_with_cache import get_data_with_cache

sys.path.insert(0, '../ml_logic')
from preprocessor import *
from data import *

In [2]:
# Import data from CSV (temp)
wingman_data = pd.read_csv("../raw_data/trimmed_data/wingman_data_v3.csv")

In [3]:
# ## Import data from GBQ
# query = f"""SELECT * FROM {GCP_PROJECT}.{BQ_DATASET}.{BQ_TABLE}"""
# data_query_cache_path = Path(LOCAL_DATA_PATH).joinpath("preclean", f"query_{DATA_SIZE}.csv")

# wingman_data = get_data_with_cache(query, data_query_cache_path)

In [4]:
# Clean
wingman_data_clean = clean_data(wingman_data)
# Preprocess
wingman_data_proc = preprocess_features_v3(wingman_data_clean)

/home/aaron/code/AaronDV17/wingman-project/baseline/../ml_logic/data.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wingman_data_cleaned.drop([
/home/aaron/code/AaronDV17/wingman-project/baseline/../ml_logic/encoders.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wingman_data_enc[''] = np.where(wingman_data_enc['type_fly'].isin(top_9_categories), wingman_data_enc['type_fly'], 'Other')
/home/aaron/code/AaronDV17/wingman-project/baseline/../ml_logic/encoders.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

## Train test split

In [5]:
X = wingman_data_proc.drop(columns=["category_no"])
y = wingman_data_proc["category_no"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

## Baseline model

In [7]:
baseline_mod = RandomForestClassifier(n_estimators=100, random_state=1, n_jobs=-1)
baseline_mod.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=1)

In [8]:
## baseline prediction

y_pred = baseline_mod.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 47.20%


## Model performance (not sure if this is needed for baseline)

In [ ]:
## confusion matrix

confusion_matrix(y_test, y_pred)

In [ ]:
## classification report

print(classification_report(y_test, y_pred))


In [ ]:
## feature importance

feature_imp = pd.Series(baseline_mod.feature_importances_, index = wingman_data.feature_names).sort_values(ascending = False)
feature_imp